In [ ]:
#Mount Google Drive with all Data to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/My Drive/liquor_sales_data/'

2012_sales.csv	2014_sales.csv	2016_sales.csv	2018_sales.csv	2020_sales.csv
2013_sales.csv	2015_sales.csv	2017_sales.csv	2019_sales.csv


In [ ]:
# Importar librerias
import pandas as pd
import numpy as np
import os 
from multiprocessing import Process, Queue
import warnings
#!pip install dataprep
#from dataprep.eda import plot, plot_missing
warnings.filterwarnings('ignore')

In [ ]:
def process_file(file_name):
    """
    Procesa la colección de datos liquor_sales_data/year_sales.csv, la cual
    contiene el registro de ventas por año.

    Returns:
        (:obj:'dataframe'): dataframe con el registro de ventas
    """
    
    # Filtro de columnas de interes
    usecols = ['Invoice/Item Number', 'Date', 'County Number',
                   'Category Name', 'Bottle Volume (ml)', 'State Bottle Cost',
                   'State Bottle Retail', 'Bottles Sold',
                   'Sale (Dollars)', 'Volume Sold (Liters)']

    # Carga datos, seleccion variables interes, indexar por fecha
    return pd.read_csv(file_name, parse_dates=['Date'],
                             usecols=usecols,
                             index_col=['Date'],
                             dtype={'County Number': 'str'}).sort_index()

        


def data_wrangling(file_name):
    """
    Realiza preparación de datos
        
    Returns:
        vendor (:obj: 'queue'): resultados de la preparación de datos para concatenar en 
                                un mismo dataframe
    """
        
    vendor = process_file(file_name)
        
    # Renombrar columnas
    vendor.columns = ['invoice_line_no', 'county_no', 'category_name',
                          'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
                          'sale_bottles', 'sale_dollars', 'sale_liters']

        # Agrupar datos en categorias simples ('categor_name')
    vendor['category_name'] = vendor['category_name'].astype(str).str.upper()
    vendor.loc[vendor['category_name'].str.contains('WHISK', na=False), 'category_name'] = 'WHISKEY'
    vendor.loc[vendor['category_name'].str.contains('VODK', na=False), 'category_name'] = 'VODKA'
    vendor.loc[vendor['category_name'].str.contains('GIN', na=False), 'category_name'] = 'GIN'
    vendor.loc[vendor['category_name'].str.contains('RUM', na=False), 'category_name'] = 'RUM'
    vendor.loc[vendor['category_name'].str.contains('NAN', na=False), 'category_name'] = 'UNKNOWN'
    searchfor = ['SCHNA', 'MALT', 'TEQU', 'LIQU', 'SHNAPP', 'PACKAG', 'COCKTAIL', 'ORDER', 'VAP', 'MEZCAL',
                     'DELISTED']
    vendor.loc[vendor['category_name'].str.contains('|'.join(searchfor), na=False), 'category_name'] = 'LIQUOR'

        # Consolidad tamaño de la botella en 750ml, 1000ml, 1750ml o otros
    vendor.loc[vendor['bottle_volume_ml'] == 750, 'bottle_volume_ml'] = '750ml'
    vendor.loc[vendor['bottle_volume_ml'] == 1000, 'bottle_volume_ml'] = '1000ml'
    vendor.loc[vendor['bottle_volume_ml'] == 1750, 'bottle_volume_ml'] = '1750ml'
    vendor['bottle_volume_ml'].loc[(vendor['bottle_volume_ml'] != '750ml') &
                                       (vendor['bottle_volume_ml'] != '1000ml') &
                                       (vendor['bottle_volume_ml'] != '1750ml')] = 'other_ml'

    return vendor
    

from multiprocessing import Pool
if __name__ == '__main__':
    path = '/content/drive/My Drive/liquor_sales_data/'
    file_list = [str(path) + "/" + file_name for  file_name in os.listdir(path)]
    
    with Pool(processes=8) as pool:
        df_list = pool.map(data_wrangling,file_list)
        combined_df = pd.concat(df_list, ignore_index=False)

In [ ]:
np.unique(combined_df.index.year)

combined_df.shape

#combined_df.to_csv('/content/drive/My Drive/liquor_sales_data/data_arima_cleaned.csv')
combined_df.columns

Index(['invoice_line_no', 'county_no', 'category_name', 'bottle_volume_ml',
       'state_bottle_cost', 'state_bottle_retail', 'sale_bottles',
       'sale_dollars', 'sale_liters'],
      dtype='object')

In [ ]:
combined_df.resample('M').mean().to_csv('/content/drive/My Drive/monthly_data_arima.csv')
